In [1]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

<IPython.core.display.Javascript object>

In [2]:
author_info = 'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [3]:
header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
response_author = requests.get(author_info,headers=header)

# capture the text content of the page from response
page_contents = response_author.text

In [4]:
# Save the page as html to avoid scrapping multiple times while experimenting
with open('author.html', 'w', encoding="utf-8") as file:
    file.write(page_contents)

# read it html page to scrape information
with open('author.html', 'r', encoding="utf-8") as file:
    html_source = file.read()

# The BeautifulSoup returns a document which we shall use to extract information
doc_author = BeautifulSoup(html_source, 'html.parser')

In [5]:
authorList = ["Name",
            "Born",
            "Died",
            "Hometown",
            "Biography",
            "Image",
            "Genre",
            "Website",
            "Influences",  
            "Rating",
            "Rating_count",
            "Review_count",
            "Works",          
            "Typical_series"
]

In [6]:
author = {key:None for key in authorList}

In [7]:
authorName = doc_author.find('h1', class_='authorName')
author['Name'] = authorName.text.strip()
author['Name']

'Suzanne Collins'

In [8]:
author_hometown = doc_author.find('div', class_='dataTitle').next_sibling
author['Hometown'] = author_hometown.text.strip()
author['Hometown']

'in  Hartford, Connecticut, The United States'

In [9]:
data = doc_author.find_all('div', class_='dataTitle')
for i in data:
    info = i.text.strip()
   
    if info == 'Website':
        i = i.find_next_sibling('div')     
        author[info] = i.find('a').get('href')
    elif info == 'Influences':
        i = i.find_next_sibling('div').find('span', attrs={'style':'display:none'}) 
        author[info] = i.text.strip()
    else:
        i = i.find_next_sibling('div')
        attr = i['class']     
        if attr == ['dataItem']:    
            author[info] = i.text.strip()
    

In [10]:
author_img = doc_author.find('div', class_='leftContainer authorLeftContainer')
author['Image'] = author_img.find('img').get('src')
author['Image']

'https://images.gr-assets.com/authors/1630199330p5/153394.jpg'

In [11]:
author_info = doc_author.find('div', class_='aboutAuthorInfo').find('span')
i = author_info.find('i')
if i != None:
    author_info.find('i').decompose()

In [12]:
author_bio = author_info.get_text(separator='\n', strip=True)
author['Biography'] = author_bio
author['Biography']

'Since 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\nThinking one'

In [13]:
author_rate = doc_author.find('span', itemprop = 'ratingValue')
author['Rating'] = author_rate.text.strip()
author['Rating']

'4.26'

In [14]:
author_rateCount = doc_author.find('span', itemprop = 'ratingCount')
author['Rating_count'] = author_rateCount.text.strip()
author['Rating_count']

'15,759,480'

In [15]:
author_rvCount = doc_author.find('span', itemprop = 'reviewCount')
author['Review_count'] = author_rvCount.text.strip()
author['Review_count']

'529,942'

In [16]:
author_data = doc_author.find('div', class_='hreview-aggregate')
for i in author_data.find_all('span'):
    check = i.get('itemprop')
    if check == 'ratingValue':
        author['Rating'] = i.text.strip()
    elif check == 'ratingCount':
        author['Rating_count'] = i.text.strip()
    elif check == 'reviewCount':
        author['Review_count'] = i.text.strip()
        author_work = i.find_next('a')
        author_book = author_work.get('href')
        author['Works'] = author_work.text.strip().split()[0]

In [20]:
author['Typical_series'] = []
typicalSeries = doc_author.find_all('div', class_='seriesDesc')
for series in typicalSeries:
    series = series.find('a', class_='bookTitle')
    series = series.text.strip()
    author['Typical_series'].extend([series])

In [18]:
genres = author['Genre'].split(',')
author['Genre'] = []
for i in genres:
    author['Genre'].extend([i])
author['Genre']

['Fiction', ' Science Fiction & Fantasy', ' Young Adult']

In [21]:
author

{'Name': 'Suzanne Collins',
 'Born': 'August 11, 1962',
 'Died': None,
 'Hometown': 'in  Hartford, Connecticut, The United States',
 'Biography': 'Since 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\nThinking one',
 'Image': 'https://images.gr-assets.com/authors/1630199330p5/153394.jpg',
 'Genre': ['Fiction', ' Science Fiction & Fantasy', ' Young Adult'],
 'Website': 'http://suzannecollinsbooks.